# Bank Deposits Forecast Model

In [ ]:
%pip install -q pymc3

In [ ]:
import validmind as vm

In [ ]:
from validmind.datasets.regression import fred_deposits as demo_dataset

deposits, fedfunds, tb3ms, gs10, gs30 = demo_dataset.load_data()

In [ ]:
vm_raw_deposits_ds = vm.init_dataset(
    input_id="raw_deposits_ds",
    dataset=deposits,
    __log=False,
)

vm_raw_fedfunds_ds = vm.init_dataset(
    input_id="raw_fedfunds_ds",
    dataset=fedfunds,
    __log=False,
)

vm_raw_tb3ms_ds = vm.init_dataset(
    input_id="raw_tb3ms_ds",
    dataset=tb3ms,
    __log=False,
)

vm_raw_gs10_ds = vm.init_dataset(
    input_id="raw_gs10_ds",
    dataset=gs10,
    __log=False,
)

vm_raw_gs30_ds = vm.init_dataset(
    input_id="raw_gs30_ds",
    dataset=gs30,
    __log=False,
)


In [ ]:
vm.tests.run_test(
    test_id="validmind.data_validation.TimeSeriesIndividualLinePlot",
    inputs={
        "datasets": [vm_raw_deposits_ds, vm_raw_fedfunds_ds, vm_raw_tb3ms_ds, vm_raw_gs10_ds, vm_raw_gs30_ds],
    }
)

In [ ]:
vm.tests.run_test(
    test_id="validmind.data_validation.TimeSeriesMissingValues",
    inputs={
        "dataset": vm_deposits_ds
    }
)

In [ ]:
vm.tests.run_test(
    test_id="validmind.data_validation.TimeSeriesFrequency",
    inputs={
        "dataset": vm_deposits_ds
    }
)

In [ ]:
vm.tests.run_test(
    test_id="validmind.data_validation.TimeSeriesFrequency",
    inputs={
        "dataset": vm_fedfunds_ds
    }
)

## Seasonality Analysis

The seasonal plot shows the deviation from the trend, so the units represent the magnitude of the seasonal effect in billions of U.S. dollars. 

The y-axis of the seasonal plot ranges from approximately -20 to +30, which suggests that the seasonal fluctuations in deposit volumes can vary by tens of billions of dollars throughout the year. These fluctuations are significant and should be considered when analyzing the data for predictive modeling or other financial analyses.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Decompose the time series
result = seasonal_decompose(df['DPSACBW027NBOG'], model='additive')

# Plot the seasonal component
seasonal = result.seasonal
fig = px.line(seasonal, title='Seasonal Component of Deposits')
fig.show()

In [ ]:
vm.tests.run_test(
    test_id="validmind.data_validation.SeasonalDecompose",
    inputs={
        "dataset": vm_raw_ds
    }
)

In [ ]:
vm.tests.run_test(
    test_id="validmind.data_validation.AutoSeasonality",
    inputs={
        "dataset": vm_raw_ds
    }
)

## Fit Bayesian Model